In [1]:
from contribution import fitness_alignment, generalization, precision_alignment

In [2]:
#importers
from pm4py import convert_to_event_log, convert_to_dataframe, format_dataframe

# Miners
from pm4py import serialize, deserialize
from pm4py import discover_dfg as dfg_discovery

from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.algo.discovery.inductive import algorithm as inductive_miner


# Evaluators
import pandas as pd
import re
import pm4py
from pm4py.discovery import DFG
from pm4py import discover_dfg_typed

In [75]:
import importlib
importlib.reload(precision_alignment)

<module 'contribution.precision_alignment' from '/opt/notebooks/contribution/precision_alignment.py'>

In [4]:
data = pd.read_csv('BPIC15_1.csv')
# log = pm4py.read_xes('BPIC15_1.xes')

/tmp/ipykernel_8725/73148813.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('BPIC15_1.csv')


In [5]:
groups = data['case:concept:name'].unique()

In [91]:
dataframe = data[data['case:concept:name'].isin(groups[:80])][:]

In [92]:
dataframe['time:timestamp'] = pd.to_datetime(dataframe['time:timestamp'], utc=True)
dataframe['concept:name'] = dataframe['concept:name'].astype(str)
dataframe['case:concept:name'] = dataframe['case:concept:name'].astype(str)

In [93]:
dfg, start_activities, end_activities = dfg_discovery(dataframe)

In [94]:
theDfg = DFG(dfg, start_activities=start_activities, end_activities=end_activities)

In [95]:
net, im, fm = pm4py.discover_petri_net_inductive(theDfg, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')

In [96]:
precision_all = precision_alignment.apply(dataframe, net, im, fm)

In [20]:
totals = precision_alignment.aggregate(precision_0_40, precision_40_80)

In [90]:
precision_all.keys()

dict_keys(['prefixes', 'prefix_count', 'start_activities', 'log_length'])

In [97]:
result = dask.delayed(precision_alignment.compute)(**precision_all, net=net, im=im, fm=fm)

In [98]:
%%memit
precision = result.compute()

[########################################] | 100% Completed | 37.46 s
peak memory: 1615.37 MiB, increment: 2.61 MiB


In [99]:
precision # Our improved parallelized way

0.016367968833723157

In [100]:
precision_alignments = pm4py.precision_alignments(dataframe, net, im, fm)

computing precision with alignments, completed variants ::   0%|          | 0/3219 [00:00<?, ?it/s]

In [101]:
precision_alignments # Pm4py result

0.016367968833723157

In [25]:
%load_ext memory_profiler